In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.options.display.max_columns = 70
pd.options.display.max_rows = 70

In [2]:
#prop.shape
good_import_cols = pd.Series(range(0,57))
#good_import_cols = good_import_cols.drop([22,32,34,49,55])


In [3]:
train_df1 = pd.read_csv("train_2016.csv", parse_dates=["transactiondate"])
prop = pd.read_csv("properties_2016.csv") #, usecols=good_import_cols)

C:\Users\board\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
missing_df = prop.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]

missing_df['missing_ratio'] = missing_df['missing_count'] / prop.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.75]

,column_name,missing_count,missing_ratio
2,architecturalstyletypeid,2979156,0.997970
3,basementsqft,2983589,0.999455
6,buildingclasstypeid,2972588,0.995769
9,decktypeid,2968121,0.994273
10,finishedfloor1squarefeet,2782500,0.932093
13,finishedsquarefeet13,2977545,0.997430
14,finishedsquarefeet15,2794419,0.936086
15,finishedsquarefeet50,2782500,0.932093
16,finishedsquarefeet6,2963216,0.992630
18,fireplacecnt,2672580,0.895272


In [5]:
train_df = train_df1.merge(prop, on='parcelid', how = 'left')

In [6]:
mean_values = train_df.mean(axis=0)
train_df_new = train_df.fillna(mean_values, inplace=True)

In [7]:
train_df['transaction_month'] = train_df['transactiondate'].dt.month
mean_values = train_df.mean(axis=0)
train_df_new = train_df.fillna(mean_values, inplace=True)

In [8]:
train_df.groupby(by=['transaction_month']).logerror.count()

transaction_month
1      6574
2      6401
3      8752
4      9407
5     10056
6     10968
7      9984
8     10510
9      9597
10     4991
11     1829
12     1742
Name: logerror, dtype: int64

In [9]:
cols_to_drop = train_df.dtypes[train_df.dtypes != "float64"].index
train_df.dtypes[train_df.dtypes != "float64"].index

Index(['parcelid', 'transactiondate', 'hashottuborspa',
       'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag',
       'taxdelinquencyflag', 'transaction_month'],
      dtype='object')

In [10]:
train_df2 = train_df.drop(['propertycountylandusecode', 'propertyzoningdesc', 'taxdelinquencyflag', 'hashottuborspa','fireplaceflag' ], axis = 1)
missing_df2 = train_df2.isnull().sum(axis=0).reset_index()
missing_df2.columns = ['count', 'val']
missing_df2[missing_df2['val'] > 0]


,count,val


In [11]:
#train_df2.to_csv("semi_clean_df.csv")

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
train_cols = train_df.drop(cols_to_drop, axis = 1).columns
train_df_scale = MinMaxScaler().fit_transform(train_df.drop(['parcelid', 'transactiondate', 'hashottuborspa',
       'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag',
       'taxdelinquencyflag', 'transaction_month', 'logerror'],axis = 1))
#train_df_scale = MinMaxScaler().fit_transform(train_df.drop(cols_to_drop, axis = 1))

In [14]:
train_y = train_df['logerror']

In [22]:
from keras.models import Sequential, Model  
from keras.layers import Dense, Activation, Dropout, Input, BatchNormalization 
from keras import losses 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
import keras.backend as K

In [16]:
def R_square(y_true, y_pred):
    SS_res = K.pow((y_true - y_pred),2)
    SS_tot = K.pow((y_true - K.mean(y_true)),2)
    return (1-(SS_res/SS_tot))

In [33]:
def create_model(learning_rate = 0.005, decay_rate = 0.001, momentum1 = 0.8, init1 = "uniform", act1 = "relu",
                act2 = "tanh"): 
    model = Sequential() 
    model.add(Dense(52, input_dim = train_df_scale.shape[1], kernel_initializer = init1, activation=act1))
    model.add(Dropout(0.2))
    model.add(Dense(100,kernel_initializer = init1, activation = act1))
    model.add(Dropout(0.5))
    model.add(Dense(26, kernel_initializer = init1, activation = act2))
    model.add(Dense(4, kernel_initializer = init1, activation = act2))
    model.add(Dense(1, kernel_initializer= "normal", activation='linear'))
    
    sgd = SGD(lr = learning_rate, momentum = momentum1, decay = decay_rate, nesterov = False )
    model.compile(loss = 'mae', optimizer = sgd, metric = [R_square])
    return model


In [36]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='mae', verbose=1, save_best_only=True, mode='max')
stopping= EarlyStopping(monitor = 'loss', patience  = 3, mode = "max")
callbacks_list = [stopping]

In [37]:
model2 = create_model().fit(train_df_scale, train_y, epochs = 20, validation_split = 0.2, batch_size = 30, 
                            callbacks = callbacks_list, verbose = 1)


C:\Users\board\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 72648 samples, validate on 18163 samples
Epoch 1/20
72648/72648 [==============================] - 4s - loss: 0.0698 - val_loss: 0.0667
Epoch 2/20
72648/72648 [==============================] - 4s - loss: 0.0697 - val_loss: 0.0670
Epoch 3/20
72648/72648 [==============================] - 4s - loss: 0.0697 - val_loss: 0.0665
Epoch 4/20
72648/72648 [==============================] - 4s - loss: 0.0697 - val_loss: 0.0666
Epoch 5/20
72648/72648 [==============================] - 4s - loss: 0.0696 - val_loss: 0.0666


In [28]:
print(model2.history.keys())

dict_keys(['loss', 'val_loss'])


In [116]:
def create_model2(epochs = 13, learning_rate = 0.001, momentum1 = 0.6, batch_size1 = 60, val_per = 0.15): 
    model = Sequential() 
    model.add(Dense(56, input_dim = train_df_scale.shape[1], init = 'uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(56,kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(52, kernel_initializer = 'uniform', activation = 'tanh'))
    model.add(BatchNormalization())
    model.add(Dense(26, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(1, kernel_initializer= 'normal'))
    decay_rate = learning_rate/epochs
    sgd = SGD(lr = learning_rate, momentum = momentum1, decay = decay_rate, nesterov = False )
    model.compile(loss = 'mae', optimizer = sgd, metric = ['R_square'])
    model.fit(train_df_scale, train_y, validation_split = val_per, epochs = epochs, batch_size = batch_size1, verbose = 1)
    return model

In [117]:
batch_model = create_model2()

C:\Users\board\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(56, input_dim=46, kernel_initializer="uniform")`
  app.launch_new_instance()
C:\Users\board\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 77189 samples, validate on 13622 samples
Epoch 1/13
6s - loss: 0.0737 - val_loss: 0.0664
Epoch 2/13
5s - loss: 0.0697 - val_loss: 0.0659
Epoch 3/13
5s - loss: 0.0697 - val_loss: 0.0665
Epoch 4/13
5s - loss: 0.0697 - val_loss: 0.0660
Epoch 5/13
5s - loss: 0.0697 - val_loss: 0.0663
Epoch 6/13
4s - loss: 0.0697 - val_loss: 0.0662
Epoch 7/13
6s - loss: 0.0697 - val_loss: 0.0660
Epoch 8/13
5s - loss: 0.0697 - val_loss: 0.0665
Epoch 9/13
5s - loss: 0.0697 - val_loss: 0.0659
Epoch 10/13
5s - loss: 0.0697 - val_loss: 0.0661
Epoch 11/13
5s - loss: 0.0696 - val_loss: 0.0667
Epoch 12/13
4s - loss: 0.0696 - val_loss: 0.0660
Epoch 13/13
5s - loss: 0.0696 - val_loss: 0.0661


In [124]:
train_df_new = pd.read_csv("train_2016.csv")
test = pd.read_csv("sample_submission.csv")
prop = pd.read_csv("properties_2016.csv")

C:\Users\board\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [127]:
test['parcelid'] = test['ParcelId']
df_test = test.merge(prop, on='parcelid', how = 'left') 
print("Test DF len: " ,len(df_test))
print("Input DF len: ", len(test))

Test DF len:  2985217
Input DF len:  2985217


In [126]:
train_cols1 = train_cols[1:len(train_cols)]

In [101]:
x_test2 = df_test[train_cols1]
test_mean_values = x_test2.mean(axis=0)
x_test1 = x_test2.fillna(test_mean_values, inplace=True)
x_test = MinMaxScaler().fit_transform(x_test1)

C:\Users\board\Anaconda3\lib\site-packages\pandas\core\generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [118]:
predictions = batch_model.predict(x_test)

C:\Users\board\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


In [121]:
sub = pd.read_csv("sample_submission.csv")
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = predictions

In [122]:
sub.to_csv('nn_starter2.csv', index=False, float_format='%.4f')

In [123]:
sub.shape

(2985217, 7)